# Detect people loitering 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uclasystem/VQPy/blob/main/examples/loitering/demo.ipynb)


## Introduction

Loitering is the activity of remaining in an area for no obvious reason. In the example [video](https://youtu.be/EuLMrUFNRxQ), we consider the region inside the garage private, and tries to detect people staying within that region for a prolonged time.

## Environment setup

Python3.8 is recommended to avoid compatibility issues when installing YOLOX.

Please download video from [here](https://youtu.be/EuLMrUFNRxQ) and place it in the same directory as this notebook, aside from running the cell below.

In [ ]:
# install YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
# download YOLOX pretrained model
!wget -bqc https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth
!cd YOLOX && pip3 install .
# download VQPy, move vqpy/ to root directory for import
!git clone https://github.com/uclasystem/VQPy.git
!mv VQPy/vqpy ./
# install VQPy's dependencies
!pip3 install lap cython_bbox shapely
import vqpy
video_path = "./loitering.mp4"	# or change to wherever you put to video
save_folder = "./vqpy_outputs"
detector_model_dir = "./"

For reference, the working directory should contain:
```
.
├── VQPy
├── YOLOX
├── loitering.mp4	# video to query on
├── vqpy		# make vqpy available for import
└── yolox_x.pth		# model checkpoint
```

## People loitering query with VQPy

### Step 1: Define a `Person` `VObj`

Since we are interested in people, first define a `Person` class. In this example, we will only be using the coordinates of each person, which are all built-in to VObj, so no extra property needs to be added.

In [ ]:
class Person(vqpy.VObjBase):
    pass

With the defined VObj type, we can filter all VObjs that's identified to be a person with:

```python
"__class__": lambda x: x == Person
```

### Step 2: Define people loitering query

#### Specify filtering conditions on `VObj` (`filter_cons`)

Trying to find people that stays in the region of interest, we filter on the coordinate of a person's base of support. This can be approximated to a built-in property of VObj: `bottom_center`, the point located at the center of the lower edge of VObj's bounding box.

Knowing where the person is, the next step is to determine if the coordinate is within the region of interest (i.e. the garage), marked by the red polygon shown below.

<img src="./demo.assets/region.png" alt="region of interest" style="zoom: 30%;" />

`vqpy.utils.within_regions([REGION, ...])` accepts a coordinate and returns `True` if it's in any of the `REGION`s.

The polygon is outlined by its 5 vertices:

In [ ]:
REGION = [(550, 550), (1162, 400), (1720, 720), (1430, 1072), (600, 1073)]

The predicate to filter VObjs in the region is then:

```python
"bottom_center": vqpy.utils.within_regions([REGION])
```

Since we wish to find person staying in the region for some duration, `vqpy.utils.continuing` could wrap the above predicate and specify the minimum time it should be satisfied.

`duration=10` sets the time threshold to 10 seconds; `name="in_roi"` can be used later to retrieve time periods of condition being satisfied.

Predicate used to filter on `bottom_center` becomes:

```python
"bottom_center": vqpy.utils.continuing(
    condition=vqpy.utils.within_regions([REGION]),
    duration=10, name="in_roi"
)
```

Combining everything we have so far, the filter condition of the query should be:

In [ ]:
filter_cons = {
    "__class__": lambda x: x == Person,
    "bottom_center": vqpy.utils.continuing(
        condition=vqpy.utils.within_regions([REGION]), duration=10, name="in_roi"
    ),
}

#### Select `VObj`'s properties for output (`select_cons`)

For each VObj that satisfies the filter conditions, we can select its properties to output. The selection is done by specifying a property name as key in the dictionary `select_cons`, where the name is either a property defined in VObj or a property from vqpy's built-in property library. The values of the dictionary are optional post-processing functions applied to value of the properties before serializing the results.

In this example, we select:

- `track_id`, tracking id, a built-in property of VObj
- `coordinate`, center coordinate of the VObj, from vqpy's built-in property library. Since the value of `coordinate` is of type `numpy.ndarray`, we need to convert it to a string with the lambda function before serializing it.

	It makes more sense to use the center of the bounding box for outputs (while `bottom_center` is the point located at center of the lower lower edge of VObj's bounding box, an approximation of where a person is standing).

- `in_roi_periods`, a list of time periods (in seconds) of VObjs satisfying the condition.

	This property is added to `Person` by wrapper `vqpy.utils.continuing`, and the name comes from appending `"_periods"` to the `name` defined in `vqpy.utils.continuing`

The select condition of the query could be:

In [ ]:
select_cons = {
    "track_id": None,
    "coordinate": lambda x: str(x),  # convert to string for JSON serialization
    # name in vqpy.continuing + '_periods' can be used in select_cons.
    "in_roi_periods": None,
}

#### Construct the query with `filter_cons` and `select_cons`

Inheriting `vqpy.QueryBase`, we can compose a query using the `filter_cons` and `select_cons` created earlier. `filename` will be used in name of the output file.

In [ ]:
class People_loitering_query(vqpy.QueryBase):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        return vqpy.VObjConstraint(
            filter_cons, select_cons, filename="loitering"
        )

### Step 3: Running the query

With the `Person` VObj and the query defined, we can run the query, where:

- `cls_name` is a tuple for mapping numerical outputs of object detector to str

	e.g. `vqpy.COCO_CLASSES` passed here starts with `("person", "bicycle", ...)`, meaning that we will map output `0` of object detector to COCO class `"person"`

- `cls_type` is a dictionary that maps name of object type (in str) to VObj types defined

	`{"person": Person}` means we wish to map COCO class `person` to VObj type `Person`

- `tasks` is a list of queries to run on the video

In [ ]:
vqpy.launch(
        cls_name=vqpy.COCO_CLASSES,
        cls_type={"person": Person},
        tasks=[People_loitering_query()],
        video_path=video_path,
        save_folder=save_folder,
        detector_name="yolox",
        detector_model_dir=detector_model_dir,
    )

## Expected result

Result of the query will be in `{save_folder}/{video_name}_{task_name}_{detector_name}.json`, output for this example should be in `./vqpy_outputs/loitering_loitering_yolox.json`.

One entry is created for each frame that has filter condition satisfied.

The 765th frame as an example:

```json
{
    "frame_id": 765,
    "data": [
      {
        "track_id": 606,
        "coordinate": "[1426.5312   492.42188]",
        "in_roi_periods": [[40, 50]]
      }
    ]
}
```

Marking person's `coordinate` with a red circle, this is

<img src="./demo.assets/marked.png" alt="with coordinate marked" style="zoom: 30%;" />

Value of `in_roi_periods` indicates that the person has been in the region in time period 00:40-00:50.